In [1]:
from dfs_service import DFS, distance_obj,filter_dups,find_best_connection_distance,MySolutionMap
from dfs_service import connection_exists,validate_connection, calculate_path_distance
from itertools import permutations
import settings
import time
from decimal import Decimal
from utils import load_maps

In [2]:
distances = distance_obj()
ride_map = load_maps()

In [3]:
 # nested loop,region to region connection
all_regions_connections = permutations([region_name for region_list, region_name in settings.REGIONS_LIST])
all_regions_connections = list(all_regions_connections)
print(f" permutations of all regions {len(list(all_regions_connections))}\n")

 permutations of all regions 362880



In [4]:
# ride_map

In [5]:
solution_map = MySolutionMap()



t0 = time.time()
for region_list, region_name in settings.REGIONS_LIST:
    t1 = time.time()
    d = DFS(region_list)
    solutions = d.start()
    filtered_solutions = filter_dups(solutions, distances)
    # solution_map[region_name] = d.start()
    solution_map[region_name] = filtered_solutions

    print(
        f" {region_name} region took {time.time() - t1}, total solutions {len(solutions)} after filter {len(filtered_solutions)}\n")

print(f" full searches took {time.time() - t0}")

 STAR_WARS_GALAXY_EDGE region took 0.0026459693908691406, total solutions 2 after filter 2

 TOON_TOWN region took 0.039473772048950195, total solutions 1728 after filter 54

 CRITTER_COUNTRY region took 0.002070903778076172, total solutions 6 after filter 6

 NEW_ORLEANS_SQUARE region took 0.0020029544830322266, total solutions 4 after filter 4

 FRONTIER_LAND region took 0.001986980438232422, total solutions 4 after filter 4

 ADVENTURE_LAND region took 0.0018987655639648438, total solutions 4 after filter 4

 MAIN_STREET region took 0.012012958526611328, total solutions 648 after filter 42

 TOMORROW_LAND region took 0.005934953689575195, total solutions 72 after filter 24

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
 FANTASY_LAND region took 30.531293869018555, total solutions 237973 after filter 24

 full searches took 30.60008716583252


In [6]:
solution_map["NEW_ORLEANS_SQUARE"]

{(69026, 19797): [69026, 6984, 76976, 72179, 19797],
 (69026, 72179): [69026, 6984, 76976, 19797, 72179],
 (72179, 69026): [72179, 19797, 76976, 6984, 69026],
 (19797, 69026): [19797, 72179, 76976, 6984, 69026]}

In [7]:
# permutation of all region connects
# we need to start in main street, so no reason to generate permutations where main street is not first
perms = permutations([name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}])
perms = list(perms)
print(len(perms))
print(f"all region names {[name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}]}")
perms = [[settings.MAIN_STREET_NAME] + list(p) for p in perms]
print(len(perms))

print(f"found {len(perms)} region to region paths")
print(f"path contains {len(perms[0]) } elements")

5040
all region names ['STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'FRONTIER_LAND', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
5040
found 5040 region to region paths
path contains 8 elements


In [8]:

# print(settings.REGIONS_LIST)

magic = ['MAIN_STREET', 'ADVENTURE_LAND', "FRONTIER_LAND", "NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"]
for idx, p in enumerate(perms):
    
    
    if (magic[0] == p[0]) and (magic[1] == p[1]) and (magic[2] == p[2]) and (magic[3] == p[3]):
        if (magic[4] == p[4]) and (magic[5] == p[5]) and (magic[6] == p[6]):
            print(p, idx)
        
        
        
    

['MAIN_STREET', 'ADVENTURE_LAND', 'FRONTIER_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND', 'TOMORROW_LAND'] 3295


In [9]:
# starting in main street,
# inside each solution for source region,check exit node
# check if exit node has connection with entry node from next region
# if none found, exit path

best_distance = float("inf")
my_inf = Decimal(float("inf"))
deadend = set()
longest_path = float("-inf")
# print(len(perms))
# print(perms[0])
best_solutions = []

FLOW_SOLUTION = []
rolling_solutions = []

for pidx, path in enumerate(perms):
    
    CURR_FLOW = []
    
    curr_path_distance = Decimal("0.0")
#     tmp_distance = 0
    
    #all exists from main entrance area
    source_valid = set(   [ e2 for (e1,e2) in solution_map[path[0] ]]    )
    rolling_steps = [source_valid, ]
    for i in range(1, len(path)):
        r_source = path[i - 1]
        r_target = path[i]
        
        if (r_source, r_target) in deadend:
            curr_path_distance = my_inf
            
            break
        
        
#         possible_sources = [ e2 for (e1,e2) in solution_map[r_source] ]
        
        
        tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
        source_valid = validate_connection(r_source, r_target, solution_map, source_valid)
       
        if len(source_valid) == 0:
            print(f"no VALID connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            break
        
        
        rolling_steps.append(source_valid)
        
        curr_path_distance += tmp_distance
        
        if tmp_distance == my_inf:
            print(f"no connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            if i >= 7:
                print(f"{path}")
            
            print("\n")
            longest_path = max(longest_path, i)
#             curr_path_distance = float("inf")
            break
            
#         else:

        
    
    if  curr_path_distance != my_inf  and (i == len(path)-1):
#     if  curr_path_distance != my_inf:
        best_distance = min(best_distance, curr_path_distance)
        if curr_path_distance == best_distance:
            best_path = path
            
        best_solutions.append( list(path) )
        rolling_solutions.append( list(rolling_steps ) )
        
#     if pidx == 3295:
#         print( "@@@", i , len(path)-1  ,path, curr_path_distance )

print(f"best distance path: {best_distance}")
print(f"longest reached: {longest_path}")
print(f"scanned: {pidx}")
print(f"best solutions: {len(best_solutions)}")

no VALID connection between MAIN_STREET STAR_WARS_GALAXY_EDGE, 1th in path 0
no VALID connection between MAIN_STREET CRITTER_COUNTRY, 1th in path 720
no VALID connection between MAIN_STREET NEW_ORLEANS_SQUARE, 1th in path 1440
no VALID connection between FANTASY_LAND TOMORROW_LAND, 7th in path 2161
no VALID connection between NEW_ORLEANS_SQUARE TOMORROW_LAND, 5th in path 2162
no VALID connection between NEW_ORLEANS_SQUARE FANTASY_LAND, 5th in path 2164
no VALID connection between CRITTER_COUNTRY ADVENTURE_LAND, 4th in path 2166
no VALID connection between CRITTER_COUNTRY TOMORROW_LAND, 4th in path 2172
no VALID connection between CRITTER_COUNTRY FANTASY_LAND, 4th in path 2178
no VALID connection between STAR_WARS_GALAXY_EDGE NEW_ORLEANS_SQUARE, 3th in path 2184
no VALID connection between STAR_WARS_GALAXY_EDGE ADVENTURE_LAND, 3th in path 2208
no VALID connection between STAR_WARS_GALAXY_EDGE TOMORROW_LAND, 3th in path 2232
no VALID connection between STAR_WARS_GALAXY_EDGE FANTASY_LAND,

In [10]:
best_solutions

[['MAIN_STREET',
  'FRONTIER_LAND',
  'STAR_WARS_GALAXY_EDGE',
  'CRITTER_COUNTRY',
  'NEW_ORLEANS_SQUARE',
  'ADVENTURE_LAND',
  'TOMORROW_LAND',
  'FANTASY_LAND'],
 ['MAIN_STREET',
  'FRONTIER_LAND',
  'STAR_WARS_GALAXY_EDGE',
  'CRITTER_COUNTRY',
  'NEW_ORLEANS_SQUARE',
  'ADVENTURE_LAND',
  'FANTASY_LAND',
  'TOMORROW_LAND']]

In [11]:
rolling_solutions[0]

[{40918, 50720, 60282, 70282, 82766, 83634, 91073},
 {60568},
 {49078},
 {714, 43753, 61323},
 {19797, 72179},
 {61114},
 {8115, 23238, 46507, 58559},
 {30923, 46167}]

In [21]:
#try to go over all PERMS and run DFS at the same time

def worm_search(curr_path_idx,  path, curr_ride_list, solutions_map, winners, distance_map):
    
    if curr_path_idx >= len(path):
        
        tot = calculate_path_distance(curr_ride_list,distance_map)
#         print(f"tot {tot}, curr_ride_list len {len(curr_ride_list)}")
        winners.append(  [tot, path, curr_ride_list]  )
        return
    
    
    #get list of (entrance, exist) nodes for current area
    for (e1, e2) in solutions_map[ path[curr_path_idx] ]:
        
        #eliminate if previous node, 
        if distance_map.get( (curr_ride_list[-1], e1) ,-1) == -1:
            continue
        
        
        
        worm_search(curr_path_idx+1,  path, curr_ride_list +solutions_map[ path[curr_path_idx] ][(e1,e2)]   , solutions_map, winners, distance_map)
        
        
        

winners = []
for pidx, path in enumerate(perms):
    #path = [tomorrowland, mainentrance ...]
    
    for (e1,e2) in solution_map["MAIN_STREET"]:
        worm_search(  0, path, solution_map["MAIN_STREET"][(e1,e2)],solution_map,winners, distances )
    
    

In [34]:
print(winners[0][0])
winners.sort(key=lambda x: x[0])
# # distances
print(winners[0][1])

print(winners[-1][0])
print(winners[-1][1])


test = {  tuple(p[1])  for p in winners  }
print(len(test))


3.205762744043298501090
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
3.506837344307342204090
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
1


In [35]:
solution_map['TOON_TOWN']

{(60515, 39868): [60515, 64430, 99677, 40788, 1924, 27887, 73844, 39868],
 (60515, 73844): [60515, 64430, 99677, 40788, 1924, 27887, 39868, 73844],
 (60515, 99677): [60515, 64430, 1924, 39868, 73844, 27887, 40788, 99677],
 (60515, 40788): [60515, 64430, 27887, 73844, 39868, 1924, 99677, 40788],
 (60515, 64430): [60515, 99677, 40788, 27887, 73844, 39868, 1924, 64430],
 (60515, 27887): [60515, 64430, 99677, 40788, 1924, 39868, 73844, 27887],
 (60515, 1924): [60515, 64430, 99677, 40788, 27887, 73844, 39868, 1924],
 (1924, 39868): [1924, 64430, 60515, 99677, 40788, 27887, 73844, 39868],
 (1924, 73844): [1924, 64430, 60515, 99677, 40788, 27887, 39868, 73844],
 (1924, 99677): [1924, 39868, 73844, 27887, 40788, 64430, 60515, 99677],
 (1924, 40788): [1924, 39868, 73844, 27887, 64430, 60515, 99677, 40788],
 (1924, 64430): [1924, 39868, 73844, 27887, 40788, 99677, 60515, 64430],
 (1924, 60515): [1924, 39868, 73844, 27887, 40788, 99677, 64430, 60515],
 (64430, 99677): [64430, 60515, 1924, 39868, 

In [14]:
### use DFS to find all combinations

all_possibilities = []

#path:   main street, tomorrowland, fantasyland ....
for pidx, path in enumerate(perms):
    
        path_possibilities = [  solution_map[path[0] ]  ]
        
        for i in range(1, len(path)):
            r_source = path[i - 1]
            r_target = path[i]
            #source, target = main street, tomorrowland
            
            #is it a dead end?
            ###
            if not connection_exists(r_source, r_target, solution_map):
                continue
            
            
            
            
            #get connections:
            #source = [  [entrance, store1 ], [entrance, store2]   ]
            #target = [  [orbital, space mountain], [ buzz, nemo]   ]
            
            path_possibilities.append( solution_map[r_target] )
        
        
        if len(path) == len(path_possibilities):
            all_possibilities.append( path_possibilities  )
        
            







In [36]:
# all_possibilities[0]
# print(len(all_possibilities))
# print(len(all_possibilities[0]))
# print((all_possibilities[0]))

In [16]:
#func

def end_end_path_build(curr_path, curr_idx, path_perms ):
    
    if curr_idx >= len(curr_path):
        return True
    
    

#pick first index, check if 

In [17]:
for posib in all_possibilities:
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-17-1eeed2dce9e7>, line 3)

In [ ]:
print(deadend)

In [ ]:
print(len(best_solutions))
print(best_path)

In [37]:

conns = (
#     ("CRITTER_COUNTRY", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "ADVENTURE_LAND"),
#          ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
#              ("MAIN_STREET", "ADVENTURE_LAND"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#     ("FRONTIER_LAND", "NEW_ORLEANS_SQUARE"),
#     ("NEW_ORLEANS_SQUARE","CRITTER_COUNTRY",),
#     ("CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE"),
#     ( "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"),
#     ("FANTASY_LAND", "TOMORROW_LAND"),
    
#         ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
#                  ("MAIN_STREET", "NEW_ORLEANS_SQUARE"),
    
    ("MAIN_STREET", "TOMORROW_LAND"),
    ("FANTASY_LAND", "TOMORROW_LAND"),
    ("FANTASY_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
    ("FANTASY_LAND", "TOON_TOWN"),


    
    
    


)

for r_source, r_target in conns:
    tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
    print(r_source, r_target,  tmp_distance, (r_source, r_target) in deadend )

MAIN_STREET TOMORROW_LAND 0.11178482480398058 False
FANTASY_LAND TOMORROW_LAND 0.08661118592832563 True
FANTASY_LAND ADVENTURE_LAND 0.11867273923671834 True
FRONTIER_LAND ADVENTURE_LAND 0.05174383191634434 True
FRONTIER_LAND CRITTER_COUNTRY 0.09229982679104619 True
NEW_ORLEANS_SQUARE CRITTER_COUNTRY 0.05021377438156316 False
NEW_ORLEANS_SQUARE STAR_WARS_GALAXY_EDGE Infinity True
FANTASY_LAND TOON_TOWN 0.07229029191445366 False


In [57]:
def find_best_toon_town_fantasy_link(solution_map, distances):
    # check each end node from region1 against all start node from region2
#     tmp_distance = Decimal(float("inf"))
    # get all solutions for source
    best_combo = None
#     resp_source, resp_target = None, None
    resp = Decimal("inf")
#     for sol_source in solution_map["FANTASY_LAND"]:
    for  f1  in settings.FANTASY_LAND: 
        for sol_target in solution_map["TOON_TOWN"]:
#             s1, s2 = sol_source[0], sol_source[-1]
#             t1, t2 = sol_target[0], sol_target[-1]
            t1,t2 = sol_target[0], sol_target[-1]

#             tmp_distance = min(distances.get((s1, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s1, t2), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t2), float("inf")), tmp_distance)
            tmp_distance1 = distances.get((f1, t1), Decimal("inf"))
                
            if tmp_distance1 == Decimal("inf"):
                continue
            
            tmp_distance2 = calculate_path_distance( solution_map["TOON_TOWN"][(t1,t2)]  ,distances )
            buff = Decimal('inf')
            f2_best = None
            buff = Decimal('inf')
            for f2 in  settings.FANTASY_LAND: 
#                 if f1 == f2:
#                     continue
                    
                tmp = distances.get( (t2, f2) , Decimal("inf"))
                buff = min(buff, tmp)
                if buff == tmp:
                    f2_best = f2
            
            if buff != Decimal("inf"):
                print(tmp_distance1,tmp_distance2,buff)
                
            tmp_distance = tmp_distance1+tmp_distance2+buff
            
            resp = min(resp, tmp_distance)
            
            if resp == tmp_distance:
                best_combo = (  f1, t1,t2,f2_best )

    return (resp, best_combo)


z = find_best_toon_town_fantasy_link(solution_map, distances)

0.07266552189434301 0.298337938230608910 0.07229029191445366
0.07229029191445366 0.298337938230608910 0.07266552189434301


In [58]:
z

(Decimal('0.443293752039405580'), (46167, 39868, 73844, 46167))

In [ ]:
for key in solution_map['FANTASY_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['FANTASY_LAND'][key]    ]}\n\n")


In [ ]:
for key in solution_map['ADVENTURE_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['ADVENTURE_LAND'][key]    ]}\n\n")